<a href="https://colab.research.google.com/github/boriskuchin/MADMO-BASE-2024/blob/main/02extra_generators_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Генераторы

Циклы позволяют удобно проходить по контейнерам и обрабатывать нужным нам образом их элементы, которые хранятся в оперативной.

Но, к сожалению, это не всегда возможно. Какие могут быть причины:
- очень большой размер файла (десятки гигабайт) - мы хотим как-либо обработать его строки и проанализировать их содержимое, но не получится загрузить в оперативную память
- не можем сразу получить необходимые данные - данные могут передаваться непрерывным потоком, и нам необходимо этот поток данных как-либо преобразовать и выделить его полезные свойства

В таких случаях нам на помощь приходит *генератор*.



**Генератор** — это объект ("тип данных"), который сразу при создании не вычисляет значения всех своих элементов. Он хранит в памяти только последний вычисленный элемент, правило перехода к следующему и условие, при котором получение новых элементов прерывается.

Вычисление следующего значения происходит лишь с помощью функции `next()`, т.е. лишь по необходимости. Предыдущее значение при этом теряется. Поэтому вычисления с помощью генераторов называются ленивыми.

В этом и заключается отличие от привычных нам контейнеров (списков, кортежей) - они хранят в памяти все свои элементы, и удалить их можно только программно.

Генераторы можно создавать несколькими способами:
- генераторные выражения
- генераторные функции
- генераторные объекты

## Генераторные выражения

Самый простой способ создания генераторов - **генераторное выражение**. Оно требует контейнера, из элементов которых будут высчитываться. По синтаксису, генераторное выражение очень похоже на list comprehension, только окружено не квадратными, а круглыми скобками:

```python
(<expression> for <item_variable> in <container>)
```

Познакомимся с ними поближе на примере генератора квадратов:

In [ ]:
genexpr = (i**2 for i in range(1, 5))

In [ ]:
type(genexpr)

generator

In [ ]:
genexpr

<generator object <genexpr> at 0x78aba2a02110>

Для получения элементов из него воспользуемся функцией `next`:

In [ ]:
next(genexpr)

1

Попробуем снова получить элемент:

In [ ]:
next(genexpr)

4

Получили следующий элемент. Попробуем и дальше получать элементы:

In [ ]:
next(genexpr)

9

In [ ]:
next(genexpr)

16

In [ ]:
next(genexpr)

StopIteration: ignored

Получили ошибку `StopIteration`. Эта ошибка является служебной, используется в цикле `for` для остановки его работы. Теперь посмотрим как генератор работает с циклом `for`:

In [ ]:
for item in (i**2 for i in range(1, 5)):
    print(item)

1
4
9
16


In [ ]:
for item in [i**2 for i in range(1, 5)]:
    print(item)

1
4
9
16


Сравним память, используемую list comprehension и генераторным выражением. Для этого нам пригодится функция `getsizeof` из модуля системных функций `sys`:

In [ ]:
from sys import getsizeof

help(getsizeof)

Help on built-in function getsizeof in module sys:

getsizeof(...)
    getsizeof(object [, default]) -> int
    
    Return the size of object in bytes.



Память, используемая list comprehension квадратов чисел до 1 000 000:

In [ ]:
getsizeof([item ** 2 for item in range(10_000_000)])

89095160

Память, используемая генераторным выражением для аналогичной задачи:

In [ ]:
getsizeof((item ** 2 for item in range(10_000_000)))

104

Как видим, экономия памяти очевидна.

## Генераторные функции

Генераторные выражения - упрощенный способ создания генераторов. Если хочется больше свободы действий, то можем воспользоваться **генераторными функциями**.

Генераторная функция отличается от обычной функции тем, что вместо команды `return` для возврата значения в ней используется `yield` - `return` завершает работу функции, а `yield` лишь приостанавливает её до следующего вызова генераторной функции.

При первом вызове метода `next()` выполняется код функции с первой команды до `yield`. При втором `next()` и последующих до конца генератора — код со следующей после `yield` команды и до тех пор, пока `yield` не встретится снова.

Рассмотрим работу функции-генератора на примере. Создадим функцию-генератор для создания квадратов чисел со смещением:

In [ ]:
def gen_fn(n):
    shift = 0
    for i in range(n):
        yield i ** 2 + shift
        shift += 1

Проверим тип созданной функции:

In [ ]:
type(gen_fn)

function

Сама по себе генераторная функция не является генератором,  но может создавать их при вызове:

In [ ]:
gen = gen_fn(5)
type(gen)

generator

Проверим как работает с нашим новым генератором цикл `for`:

In [ ]:
for i in gen:
    print(i)

0
2
6
12
20


С помощью генераторных функций можно создавать бесконечные генераторы, которые могут порождать новые элементы без остановки.

Создадим генератор, который возвращает кубы натуральных чисел:

In [ ]:
def cubes():
    i = 0
    while True:
        yield i ** 3
        i += 1

Можем проверить его работу, запустив в цикле, но предварительно добавить счетчик для ручной остановки цикла:

In [ ]:
counter = 0
for item in cubes():
    print(item, end=' ')
    counter += 1
    if counter == 15:
        break

0 1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 

## Примеры

In [ ]:
def gen_fn1():
    print("Reached 1st yield")
    yield 1
    print("Reached 2nd yield")
    yield 2
    print("Reached 3rd yield")
    yield 3
    print(4)

In [ ]:
for i in gen_fn1():
    print(i)
    print()

Reached 1st yield
1

Reached 2nd yield
2

Reached 3rd yield
3

4


In [ ]:
gen = gen_fn1()

In [ ]:
next(gen)

Reached 1st yield


1

In [ ]:
def read_file(filename):
    with open(filename) as f:
        while True:
            line = f.readline()
            if line == "":
                break
            yield line

In [ ]:
read_file("text.txt")

'aaa\n'

In [ ]:
filename = "text.txt"
with open(filename) as f:
    while True:
        line = f.readline()
        if line == "":
            break
        print(line)

aaa

bbb

ccc

ddd

123

1231

1244



In [ ]:
for line in read_file("text.txt"):
    print(line)

aaa

bbb

ccc

ddd

123

1231

1244



In [ ]:
gen = read_file("text.txt")

In [ ]:
try:
    print(next(gen))
except StopIteration:
    print("generator done!")

generator done!


In [ ]:
%%file text.txt
aaa
bbb
ccc
ddd
123
1231
1244

Writing text.txt


# Чтение и запись в файл

Создадим простой файл с помощью unix-команд:

In [ ]:
%%file text.txt
abcd
efgh
ijkl

Writing text.txt


In [ ]:
!cat text.txt

abcd
efgh
ijkl


## Чтение файла

Для открытия фалы используем команду `open`. Она принимает 2 аргумента - путь до файла и режим (по-умолчанию, `'r'`):

In [ ]:
f = open('text.txt', 'r')
f

<_io.TextIOWrapper name='text.txt' mode='r' encoding='UTF-8'>

In [ ]:
type(f)

_io.TextIOWrapper

In [ ]:
help(open)

Help on built-in function open in module io:

open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)
    Open file and return a stream.  Raise OSError upon failure.
    
    file is either a text or byte string giving the name (and the path
    if the file isn't in the current working directory) of the file to
    be opened or an integer file descriptor of the file to be
    wrapped. (If a file descriptor is given, it is closed when the
    returned I/O object is closed, unless closefd is set to False.)
    
    mode is an optional string that specifies the mode in which the file
    is opened. It defaults to 'r' which means open for reading in text
    mode.  Other common values are 'w' for writing (truncating the file if
    it already exists), 'x' for creating and writing to a new file, and
    'a' for appending (which on some Unix systems, means that all writes
    append to the end of the file regardless of the current seek position

Получился объект `f` одного из файловых типов. Что с ним можно делать? Можно его использовать в `for` цикле, каждый раз будет возвращаться очередная строка файла (включая `'\n'` в конце; в конце последней строки текстового файла `'\n'` может и не быть).

In [ ]:
for i, s in enumerate(f):
    print(i, s)

0 abcd

1 efgh

2 ijkl



После работы с файлом его необходимо закрыть:


In [ ]:
f.close()

In [ ]:
f.closed

True

Общий формат работы с файлом выглядит так:

In [ ]:
# open()
# work with file
# close()

Поэтому часто при работе с файлами используют менеджеры контекстов:

In [ ]:
file_strings = []
with open('text.txt') as f:
    for s in f:
        print(s[:-1])
        file_strings.append(s)

abcd
efgh
ijkl


In [ ]:
file_strings

['abcd\n', 'efgh\n', 'ijkl\n']

У объекта `f` имеются и полезные методы:

Метод `f.read(n)` читает `n` символов (когда файл близится к концу и прочитать именно `n` символов уже невозможно, читает меньше; в самый последний раз он читает 0 символов и возвращает `''`). Прочитаем файл по 2 символа.

In [ ]:
with open('text.txt') as f:
    while True:
        c = f.read(2)
        if c == '':
            break
        else:
            print(c.__repr__())

'ab'
'cd'
'\ne'
'fg'
'h\n'
'ij'
'kl'
'\n'


Если не указать количество символов, то файл прочитается полностью:

In [ ]:
with open('text.txt') as f:
    s = f.read()

In [ ]:
s

'abcd\nefgh\nijkl\n'

Если хотим прочитать файл построчно, то пользуемся методом `readline`:

In [ ]:
with open('text.txt') as f:
    count = 0
    while True:
        s = f.readline()
        if s == '':
            break
        else:
            print(f'count: {count}, {s.__repr__()}')
            count += 1

count: 0, 'abcd\n'
count: 1, 'efgh\n'
count: 2, 'ijkl\n'


In [ ]:
with open('text.txt') as f:
    s1 = f.readline()[:-1]
    s2 = f.readline()[:-1]
    s3 = f.readline()[:-1]

print(s1, s2, s3, sep='\n')

abcd
efgh
ijkl


Метод `f.readlines()` возвращает список строк (его лучше не применять для очень больших файлов).

In [ ]:
with open('text.txt') as f:
    lst_of_words = f.readlines()
lst_of_words

['abcd\n', 'efgh\n', 'ijkl\n']

Теперь посмотрим, чем же оператор `with` лучше, чем пара `open` - `close`.

In [ ]:
def a(name):
    global f
    f = open(name)
    s = f.readline()
    n = 1 / 0
    f.close()
    return s

In [ ]:
a('text.txt')

ZeroDivisionError: ignored

In [ ]:
f.closed

False

In [ ]:
f.close()
f.closed

True

In [ ]:
f = open("text.txt")

Произошло исключение, мы покинули функцию до строчки `close`, и файл не закрылся.

In [ ]:
def a(name):
    global f
    with open(name) as f:
        print(f'File is opened: {not f.closed}')
        s = f.readline()
        n = 1 / 0
    return s

In [ ]:
a('text.txt')

File is opened: True


ZeroDivisionError: ignored

In [ ]:
f.closed

True

Теперь всё в порядке.

## Запись в файл

Чтобы открыть файл на запись, нужно указать в качестве второго аргумента `'w'`:

In [ ]:
f = open('newtext.txt', 'w')

In [ ]:
f.write('aaa\n')

4

In [ ]:
f.write('bbb\n')

4

In [ ]:
f.write('ccc\n')

4

In [ ]:
f.close()

In [ ]:
!cat newtext.txt

aaa
bbb
ccc


In [ ]:
with open('newtext.txt', 'w') as f:
    f.write('aaa\n')
    f.write('bbb\n')
    f.write('ccc\n')

In [ ]:
!cat newtext.txt

aaa
bbb
ccc


Создадим функцию, которая копирует старый текстовый файл в новый. Если строки нужно как-нибудь обработать, в последней строчке вместо `line` будет стоять что-нибудь вроде `f(line)`:

In [ ]:
def copy(old_name, new_name):
    with open(old_name) as old, open(new_name, 'w') as new:
        for line in old:
            new.write(line)

In [ ]:
def copy(old_name, new_name):
    with open(old_name) as old:
        with open(new_name, 'w') as new:
            for line in old:
                new.write(line)

In [ ]:
copy('text.txt', 'newtext.txt')

In [ ]:
!cat newtext.txt

abcd
efgh
ijkl


# f-строки

**f-строки** - более новый и удобный способ форматирования строк, добавлен в Python 3.6:

In [ ]:
year = 2023
season = 7
f'В {year}-м году состоится {season}-й сезон курса Python 3' # Обратите внимание на символ f перед строкой

'В 2023-м году состоится 7-й сезон курса Python 3'

f-строки поддерживают форматирование чисел:

In [ ]:
year = 2023
season = 7

# .2f - вещественное число с двумя знаками после запятой
f'В {year}-м году состоится {season: .2f}-й сезон курса Python 3'

'В 2023-м году состоится  7.00-й сезон курса Python 3'

Внутри f-строк можно выполнять различные операции:

In [ ]:
year = 2023

f'В {year}-м году состоится {year-2017+1}-й сезон курса Python 3'

В 2023-м году состоится 7-й сезон курса Python 3


Можно обращаться к элементам списков по индексу:

In [ ]:
years = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
season = 7

f'В {years[season]}-м году состоится {season}-й сезон курса Python 3'

В 2023-м году состоится 7-й сезон курса Python 3


И даже использовать функции и методы:

In [ ]:
year = 2023
season = 7
name = 'Python 3'

f'В {year}-м году состоится {season}-й сезон {name.upper()}'

В 2023-м году состоится 7-й сезон PYTHON 3


# Исключения

Программа в Python падает (останавливает свое выполнение) сразу как только встречает ошибку. Ошибки в Python делятся на две категории:
- синтаксические ошибки
- исключения

**Синтаксические ошибки** возникают, когда интерпретатор (а точнее его компонент - парсер) обнаруживает некорректное выражение, например:

In [ ]:
print(0 / 0

SyntaxError: ignored

Стрелочка показывает, где именно парсер наткнулся на ошибку. В примере выше указана лишняя скобка. Уберем и запустим ячейку еще раз:

In [ ]:
print(0 / 0)

ZeroDivisionError: ignored

In [ ]:
a = 1
b = 0
print(a / b)

ZeroDivisionError: ignored

В этот раз мы столкнулись с **исключением**. Этот тип ошибки возникает, когда синтаксически верный код приводит к ошибке. Последняя строка показывает с исключением какого типа мы столкнулись - в нашем примере это `ZeroDivisionError` (ошибка деления на ноль).

В Python находится огрномное количество встроенных исключений, но если вам их не хватит, то можно создать их и самому.

В примере выше полученное исключение означало, что мы пытаемся делить на ноль, а эта операция запрещена. Порой в написанных нами программах тоже возникает необходимость проверять код на корректность значений, хранящихся в переменной.

Например, наша функция корректно работает только с положительными числами, поэтому нужно остановить выполнение программы, если пользователь подал на вход ей неположительное число.

Для таких случаев в Python есть команда `raise`:

In [ ]:
from math import sqrt

def cool_func(x):
    print("Running cool func...")
    if x >= 0:
        result = sqrt(x) + x
    else:
        raise Exception('x must be larger than zero! The value of x was '+ str(x))
    print("Result:", result)
    return result

In [ ]:
cool_func(9)

Running cool func...
Result: 12.0


12.0

In [ ]:
cool_func(-9)

Running cool func...


Exception: ignored

In [ ]:
def cool_func(x):
    if x < 0:
        raise Exception('x must be larger than zero! The value of x was '+ str(x))

    print("Running cool func...")
    result = sqrt(x) + x
    print("Result:", result)
    return result

В нашем случае мы проверяем значение аргумента x на корректность в середине тела нашей функции, но мы бы могли сделать это и в начале, добавив отдельное условие `if`. На практике такие проверки типа:

```python
if condition:
    raise Exception
```

случаются очень часто. И для такого применения в Python есть отдельная команда - `assert`:

In [ ]:
def cool_func(x):
    assert type(x) is int or type(x) is float, "Bad type"
    assert x >= 0, 'x must be larger than zero! The value of x was '+ str(x)
    assert x != 1024, 'we don\'t like number 1024!'

    print("Running cool func...")
    return sqrt(x)

In [ ]:
cool_func(9)

Running cool func...


3.0

In [ ]:
cool_func(-9)

AssertionError: ignored

In [ ]:
cool_func(1024)

AssertionError: ignored

In [ ]:
cool_func('9')

AssertionError: ignored

In [ ]:
def integrate(a, b):
    """f=1 integrate"""
    assert type(a) is int or type(a) is float, 'argument a must be int or float!'
    assert type(b) is int or type(b) is float, 'argument b must be int or float!'

    return b - a

In [ ]:
integrate(1,'2')

AssertionError: ignored

`assert` всегда вызывает AssertionError при невыполнении условия.

## Конструкция `try-except`

На практике при работе с разными библиотеками исключения будут возникать часто. И не всегда мы бы хотели, чтобы при этом программа прекращала свое выполнение - иногда мы можем написать правила обработки возникающих ошибок.

Например, подключение к некоторым серверам может быть непостоянным, и при подключении к базе данных с первого раза может не получиться подключиться. При невозможности подключения библиотека по работе с базами данных почти наверняка выдаст исключение, сообщающее что подключиться не удалось. Обычно при этом программа упадет. Но нам бы хотелось подождать немного и попробовать подключиться еще раз, если с первого раза не получилось.

В подобных ситуациях на помощь приходит конструкция `try-except`:

```python
try:
    <блок команд, откуда хотим ловить ошибки>
except <исключение 1>:
    <блок команд, который выполнится, если поймаем исключение 1>
except <исключение 2>:
    <блок команд, который выполнится, если поймаем исключение 2>
else:
    <блок команд, который выполнится, если исключений не поймали>
finally:
    <блок команд, который выполнется в любом случае>
```

Рассмотрим пример:

In [ ]:
temp_list = [1, 2, 3]

In [ ]:
try:
    #cool_func(-9)
    #print(1 / 0)
    #print(temp_list[10])
    #print({"key": "value"}["other_key"])
    print(1+2)
except (AssertionError, IndexError):
    print("Assert / Index failed!")
except ZeroDivisionError:
    print("Division by zero!")
except:
    print("Something bad happened!")
else:
    print("Everything is fine!")
finally:
    print("Always print this!")

print("Out of try-except")
print(2**10)

3
Everything is fine!
Always print this!
Out of try-except
1024


**Вопрос.** Зачем нужен `finally`? Чем отличаются следующие два фрагмента?

```python
try:
    raise Exception
except Exception:
    pass
print('finally')
```

и

```python
try:
    raise Exception
except Exception:
    pass
finally:
    print('finally')
```

**Ответ.** `finally` выполнит свой код **в любом случае**, например, если в `except` вылезло новое исключение.

In [ ]:
try:
    k = 1 / 0
except:
    k = 0
    #raise Exception
    temp_list[10] = k
finally:
    print(k)
print(42)

0


IndexError: ignored

In [ ]:
try:
    print(42)
    print(temp_list[42])
    print(54)
except BaseException as e:
    print(e, type(e))
    print(dir(e))
    print(e.__traceback__)
    print(dir(e.__traceback__))
    print(e.__traceback__.tb_lineno)

42
list index out of range <class 'IndexError'>
['__cause__', '__class__', '__context__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__suppress_context__', '__traceback__', 'args', 'with_traceback']
['tb_frame', 'tb_lasti', 'tb_lineno', 'tb_next']
3
